## Lista 2 - Aprendizagem de máquina probabilístico
- Aluno: Lucas Rodrigues Aragão - Graduação 538390

In [2]:
import pandas as pd
import numpy as np

## Modelos

### Regressão linear bayesiana

#### Estimação 
1. Definir, a partir de conhecimentos anteriores, momentos da priori $p(w) = \mathcal{N}(w|m_0,S_0)$ e a variância do ruído, $p(\epsilon) = \mathcal{N}(\epsilon|0, \sigma^2)$

2. A partir de $\mathcal{D}=(X,y)$, calcular a posteriori de w
    - $P(w|D) = \mathcal{N}(w|\mu,\Sigma)$, em que
    - $\mu = m_0 + (S_0 X^T+ \sigma^2I)^{-1}S_0X^T(y-X m_0)$ 
    - $\Sigma = S_0 - (S_0 X^T X + \sigma^2 I)^{-1} S_0 X^T X S_0$
3. Retornar a posteriori dos parâmetros $p(w|D)$

#### Predição
1. A partir dos dados $X_{\ast} \in \mathbb{R}^{(N_{\ast} \times D)}$, retornar a distribuição 

$$p(y_{\ast}| X_{\ast}) = \mathcal{N}(y_{\ast}|X_{\ast}\mu, X_{\ast} \Sigma X_{\ast}^T + \sigma^2I)$$


In [ ]:
# TODO: ajeitar os valores dos hiperparametros iniciais
class BayesianLinReg():
    def __init__(self, sigma2,m0,s0):
        self.sigma2 = sigma2
        self.m0 = m0
        self.s0 = s0 
        
    def estimate(self, X, y):

        A_inv = np.linalg.inv(self.s0 @ X.T @ X + self.sigma2 * np.eye(X.shape[0])) # termo comum nas duas contas
        self.mu = self.m0 + A_inv @ self.s0 @ X.T @ (y - X @ self.m0)
        self.Sigma = self.s0 - A_inv @ self.s0 @ X.T @ X @ self.s0
        
        posteriori  = np.random.multivariate_normal(self.mu.flatten(), self.Sigma) # N(w|self.mi, self.sigma)
        return posteriori
    
    def predict(self, X_ast):
        media = X_ast @ self.mu 
        var = X_ast @ self.Sigma @ X_ast.T + (self.sigma2 * np.eye(X_ast.shape[0]))

        predictions = np.random.multivariate_normal(media, var)
        return predictions

In [8]:
linear_regression_data = pd.read_csv("linear_regression_data.csv")
X = linear_regression_data["col1"]
y = linear_regression_data["col2"]

### Regressão Polinomial Bayesiana

As expressões do modelo continuam as mesmas.
As mudanças são que, agora temos,

$$\Phi = \phi(X) = [\phi(x_1), \phi(x_2), \cdots, \phi(x_N)]^T$$
em que $\phi(x_i) = [1,x_i, x_i^2, \cdots, x_i^P]^T$, onde $P$ é a ordem do polinômio desejado

E com isso nosso $y$ será definido por $y= \Phi w + \epsilon$. Dessa forma o cálculo de $\mu$ e $\Sigma$ seram dados por 

- $\mu = m_0 +(S_0 \Phi^T \Phi + \sigma^2I)^{-1}S_0 \Phi^T (y - \Phi m_0) $  
- $\Sigma = S_0 +(S_0 \Phi^T \Phi + \sigma^2I)^{-1}S_0 \Phi^T \Phi S_0$

E a nossa predição é dada por 

$$p(y_\ast| X_\ast, \mathcal{D}, m_0, S_0, \sigma^2) = \mathcal{N}(y_\ast|\Phi_\ast \mu , \Phi_\ast \Sigma \Phi_\ast^T + \sigma^2I)$$

In [5]:
class BayesianPolyReg():
    def __init__(self, sigma2, m0, s0, degree):
        self.sigma2 = sigma2
        self.m0 = m0
        self.s0 = s0
        self.degree = degree
    
    def elevate(self, X: pd.DataFrame):
        x_vals = X.values.flatten()  
        Phi = np.vstack([x_vals**d for d in range(self.degree + 1)]).T        
        return Phi

    def estimate(self, X,y):
        Phi = elevate(X)
        A_inv = np.linalg.inv(self.s0 @ Phi.T @ Phi + self.sigma2 * np.eye(Phi.shape[0])) # termo comum nas duas contas
        self.mu = self.m0 + A_inv @ self.s0 @ X.T @ (y - Phi @ self.m0)
        self.Sigma = self.s0 - A_inv @ self.s0 @ Phi.T @ Phi @ self.s0
        
        posteriori  = np.random.multivariate_normal(self.mu.flatten(), self.Sigma) # N(w|self.mi, self.sigma)
        return posteriori
    
    def predict(self, X_ast):
        # fazer a transformacao do X_ast em Phi_ast
        Phi_ast = elevate(X_ast)
        media = Phi_ast @ self.mu 
        var = Phi_ast @ self.Sigma @ Phi_ast.T + (self.sigma2 * np.eye(Phi_ast.shape[0]))

        predictions = np.random.multivariate_normal(media, var)
        return predictions


### Regressão logística IRLS

#### Estimação
1. Definir, a partir de conhecimentos anteriores, os momentos da priori $p(w) = \mathcal{N}(w|m_0,S_0)$ e o valor inicial de $m_0 \in \mathbb{R}^D$.

2. A partir dos dados repetir até convergência:
    $$w_t = w_{t-1} + A^{-1}\big[X^T(y - \sigma(Xw_{t-1})) - S_0^{-1}(w_{t-1} - m_0)]$$
    - Em que
        - $A =X^TR_{t-1}X + S_0^{-1}$
        - $R_{t-1} = \text{diag}([R_{t-1}]_{11}, \cdots, [R_{t-1}]_{NN})$
        - $[R]_{ii} = \sigma(w^T_{t-1} x_i) (1 - \sigma(w^T_{t-1} x_i))$

3. Retornar os parâmetros $\hat{w}$

#### Predição 

1. Dado $x_{\ast}$, a predição é dada por:
    $$p(y_{\ast} = 1| x_\ast) = \sigma(\hat{w}^Tx_{\ast})$$

In [ ]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

class LogisIRLS():
    def __init__(self, m0, s0, w0):
        self.m0 = m0
        self.s0 = s0
        self.W = w0

    def estimate(self, X:pd.DataFrame, y, epochs, conv_lim):
        i = 0
        dif = np.inf()
        n = X.shape[0]    
        inv_s0 = np.linalg.inv(self.s0)
        R = np.zeros(shape= (n,n))

        while (i<epochs or dif< conv_lim):
            # preencher a matriz R 
            old_w = self.W
            for index, row in X.iterrows():
                sigm = sigmoide(old_w.T @ row)
                R[index][index] = sigm  * (1 - sigm)    

            # calcular a matriz A
            A = X.T @ R @ X + inv_s0
            # atualizar os pesos 
            next_w = old_w + np.linalg.inv(A) @ (X.T @ (y- sigmoide(X @ old_w))) - inv_s0 @ (old_w- self.m0)
            self.W = next_w

            # calcular criterios de parada
            dif = np.abs(old_w - next_w)
            i += 1

        

    def predict(self, X_ast):
        return sigmoide(self.W.T @ X_ast)

### Regressão logística Bayesiana

#### Estimação 
- Passos 1 a 3 do IRLS para encontrar o $\hat{w}$

- Aproximar a posteriori de $w$

$$p(w|\mathcal{D}) \approx \mathcal{N}(w|\hat{w}, H^{-1})$$

- Em que 
    - $H = X^T\hat{R}X + S_0^{-1}$
    - $\hat{R} = \text{diag} (\hat{R}_{11}, \cdots, \hat{R}_{NN})$
    - $\hat{R}_{ii} = \sigma(\hat{w}^T x_i)(1- \sigma(\hat{w}^T x_i))$

#### Predição 

1. Dado $x_{\ast}$, retornar a distribuição preditiva

- Usando **Monte Carlo**: 
$$p(y_{\ast}|x_{\ast} \approx \frac{1}{S} \sum^S_{s=1} \sigma(w_s^Tx_{\ast}))$$

- Onde $w_s \sim \mathcal{N}(w|\hat{w}, H^{-1})$

- Usando **probit**

$$p(y_\ast = 1|x_\ast) \approx \sigma((1+ \pi \sigma^w_a/8)^{1/2} \mu_a)$$

- Onde $\mu_a = w^T x_\ast$ e $\sigma^2_a = x_\ast^T H^{-1}x_\ast$

In [ ]:
class BayesianRegLogis():
    def __init__(self, m0, s0, w0):
        self.m0= m0
        self.s0 = s0
        self.W = w0
    
    def estimate(self, X:pd.DataFrame, y, epochs, conv_lim):
        irls = LogisIRLS(m0= self.m0, s0= self.s0,w0= self.W)
        irls.estimate(X = X, y = y, epochs=epochs, conv_lim= conv_lim)
        self.W = irls.W
        w_hat = self.W
        n = X.shape[0]
        R_hat = np.zeros((n,n))
        
        for index, row in X.iterrows():
            sigm = sigmoide(w_hat.T @ row)
            R_hat[index][index] = sigm  * (1 - sigm)    
        
        H = X @ R_hat @ X + np.linalg.inv(self.s0)
        self.H = H

        #aproximar a  posteriori de w
        posteriori = np.random.multivariate_normal(w_hat, np.linalg.inv(H))
        self.posteriori_hat = posteriori

    def predict(self, X_ast, approx = "mc"):
        if approx == "mc":
            # TODO: Implementar a aproximacao de monte carlos
            #aproximacao de monte carlo
            pass
        else:
            #aproximacao de probit
            mu_a = self.posteriori_hat @ X_ast
            sigma_a = X_ast.T @ np.linalg.inv(self.H) @ X_ast
            pred_1 = np.sqrt(1 + np.pi()*sigma_a/8)
            predictions = sigmoide(pred_1 * mu_a)
        return predictions

    

## Questões

### Questão 1